In [1]:
""" Proof of concept for the random-cut-hyperplanes idea """
import sys
import numpy as np
from scipy.stats import scoreatpercentile
from sklearn.metrics import confusion_matrix

In [2]:
# sys.setrecursionlimit(50000)

# Steps for the algorithm
#
# Given a dataset, X:
#   1. Get the feature ranges for each feature within X.
#   2. Sample a random point between each feature.
#   3. Create a hyperplane using each of these points.
#   4. Get the normal to that plane.
#   5. Determine which side of the plane the point lies on.
#   6. Repeat for the two sides.
#   7. If an individual point is left, stop iteration
#
class IsolationForest(object):
    def __init__(self, n_estimators=10):
        self.n_estimators = n_estimators

    def fit(self, points):
        self.trees = []
        for i in range(self.n_estimators):
            self.trees.append(IsolationTree(points))
        return self

    def decision_function(self, points):
        depths = np.array([tree.decision_function(points) for tree in self.trees])
        mean_depths = np.mean(depths, axis=0)
        # Normalize the points
        scores = 2**-(mean_depths / average_path_length(points.shape[0]))
        return scores

    def get_depths(self, points):
        depths = np.array([tree.decision_function(points) for tree in self.trees])
        mean_depths = np.mean(depths, axis=0)
        return mean_depths


class IsolationTree(object):
    def __init__(self, points, group_threshold=15, depth=1):
        self.child_left = self.child_right = None
        self.points = points
        self.group_threshold = group_threshold
        self.num_points = self.points.shape[0]
        self.split(self.points, depth)

    def __str__(self):
        return f"<{self.__class__.__name__} num_points:{self.num_points}>"

    @property
    def is_leaf(self):
        return self.child_left == None and self.child_right == None

    def split(self, points, depth=1):
        node, points_left, points_right = self.get_split(points)

        if not node or depth >= 50:
            return self

        self.node = node
        self.child_left = IsolationTree(points_left, depth=depth + 1)
        self.child_right = IsolationTree(points_right, depth=depth + 1)

    def decision_function(self, points):
        return np.array([self.get_depth(point) for point in points])

    def get_depth(self, point):
        if self.is_leaf:
            return 1
        else:
            if self.node.is_point_left(point):
                return 1 + self.child_left.get_depth(point)
            else:
                return 1 + self.child_right.get_depth(point)

    def get_split(self, points):
        if self.num_points <2:
            return (None, None, None)
        split_feature = sample_feature(points)
        split_threshold = sample_split_threshold(points, split_feature)
        node = Node(split_threshold, split_feature)

        positions = node.position_of_points(points)

        points_left = points[positions]
        points_right = points[np.logical_not(positions)]

        return (node, points_left, points_right)


class Node(object):
    def __init__(self, threshold, feature):
        self.threshold = threshold
        self.feature = feature

    def is_point_left(self, point):
        return point[self.feature] < self.threshold

    def position_of_points(self, points):
        return np.array([self.is_point_left(point) for point in points])
    
def get_feature_ranges(points):
    """ Yields the min and max of each feature in a vector. """
    points_transpose = np.transpose(points)
    for point in points_transpose:
        yield (np.min(point), np.max(point))
        
def sample_feature(point):
    length = point.shape[-1]
    return np.random.randint(low=0, high=length)

def sample_split_threshold(points, feature):
    return list(generate_point(points))[feature]

def average_path_length(n):
    return 2 * harmonic_approx(n - 1) - (2 * (n - 1) / n)

def harmonic_approx(n):
    return np.log(n) + 0.5772156649

class RandomHyperplanes(object):
    def __init__(self, n_estimators=10):
        self.n_estimators = n_estimators

    def fit(self, points):
        self.planes = []
        for i in range(self.n_estimators): 
            self.planes.append(HyperplaneCollection(points))
        return self

    def decision_function(self, points):
        depths = np.array([plane.decision_function(points) for plane in self.planes])
        mean_depths = np.mean(depths, axis=0)
        # Normalize the points
        scores = 2**-(mean_depths / average_path_length(points.shape[0]))
        return scores

    def get_depths(self, points):
        depths = np.array([plane.decision_function(points) for plane in self.planes])
        mean_depths = np.mean(depths, axis=0)
        return mean_depths

In [3]:
def _gen_hard_data(n, p, infection_pct, variance=10.0, mu=5.0):
    X = np.random.randn(n, p)

    # hard data
    # Weight it to the number of features
    is_anomaly = np.random.rand(n, p) < (infection_pct / p)
    X[is_anomaly] = variance * np.random.randn() + mu

    y = np.zeros(shape=(n,))

    tmp = np.array([np.any(r) for r in is_anomaly])
    y[tmp] = 1.0

    return (X, y)

def _gen_easy_data(n, p, infection_pct, variance=10.0, mu=5.0):
    X = np.random.randn(n, p)
    is_anomaly = np.random.choice(n, size=int(infection_pct*n), replace=False)
    X[is_anomaly] = variance * np.random.randn(is_anomaly.shape[0], p) + mu
    y = np.zeros(shape=(n,))
    y[is_anomaly] = 1.0
    return (X, y)

In [4]:
def run_plane_simul(points, y):
    print("Beginning plane fit...")
    rhp = RandomHyperplanes(n_estimators=N_ESTIMATORS)
    rhp = rhp.fit(points)
    print("done fitting")

#     scores = rhp.decision_function(points)
#     threshold = scoreatpercentile(scores, 100 - SCORE_AT)
#     anomalies = scores >= threshold
#     y_pred = np.zeros(shape=anomalies.shape)
#     y_pred[anomalies] = 1
    
#     """
#     correct_guesses = np.count_nonzero(y[np.where(scores <= threshold)])
#     incorrect_guesses = y[np.where(scores <= threshold)].shape[0] - \
#         correct_guesses

#     print("Correct guesses:", correct_guesses)
#     print("Incorrect guesses:", incorrect_guesses)
#     print("Expected", np.count_nonzero(y), "anomalies")
#     """
#     cnf_matrix = confusion_matrix(y, y_pred)

#     """
#     tn, fp, fn, tp = cnf_matrix.ravel()
#     print(f"tp: {tp} \ntn: {tn} \nfp: {fp} \nfn: {fn}")
#     """
#     cnf_matrix = cnf_matrix.astype('float') / \
#         cnf_matrix.sum(axis=1)[:, np.newaxis]

#     """
#     tn, fp, fn, tp = cnf_matrix.ravel()
#     print(f"Normalized \ntp: {tp} \ntn: {tn} \nfp: {fp} \nfn: {fn}")
#     """
#     print(cnf_matrix)

    depths = rhp.get_depths(points)
    anomalous_depths = depths[np.where(y==1.0)]
    non_anomalous_depths = depths[np.where(y==0.0)]    
    print("Average anomalous depth:", np.mean(anomalous_depths))
    print("Average non-anomalous depth:", np.mean(non_anomalous_depths))
    return (None, depths, None, y)


def run_iforest_simul(points, y):
    print("Beginning iforest fit...")
    iforest = IsolationForest(n_estimators=N_ESTIMATORS)
    iforest = iforest.fit(points)
    print("done fitting")

#     scores = iforest.decision_function(points)
#     threshold = scoreatpercentile(scores, 100 - SCORE_AT)
#     anomalies = scores >= threshold
#     y_pred = np.zeros(shape=anomalies.shape)
#     y_pred[anomalies] = 1

#     """
#     correct_guesses = np.count_nonzero(y[np.where(scores <= threshold)])
#     incorrect_guesses = y[np.where(scores <= threshold)].shape[0] - \
#         correct_guesses

#     print("iforest Correct guesses:", correct_guesses)
#     print("iforest Incorrect guesses:", incorrect_guesses)
#     print("Expected", np.count_nonzero(y), "anomalies")
#     """
#     iforest_cnf_matrix = confusion_matrix(y, y_pred)
#     """
#     tn, fp, fn, tp = iforest_cnf_matrix.ravel()
#     print(f"tp: {tp} \ntn: {tn} \nfp: {fp} \nfn: {fn}")
#     """
#     iforest_cnf_matrix = iforest_cnf_matrix.astype('float') / \
#             iforest_cnf_matrix.sum(axis=1)[:, np.newaxis]

#     print(iforest_cnf_matrix)

#     """
#     tn, fp, fn, tp = iforest_cnf_matrix.ravel()
#     print(f"Normalized \ntp: {tp} \ntn: {tn} \nfp: {fp} \nfn: {fn}")
#     """
    depths = iforest.get_depths(points)
    anomalous_depths = depths[np.where(y==1.0)]
    non_anomalous_depths = depths[np.where(y==0.0)]    
    print("Average anomalous depth:", np.mean(anomalous_depths))
    print("Average non-anomalous depth:", np.mean(non_anomalous_depths))
    return (None, depths, None, y)

In [24]:
class HyperplaneCollection(object):
    def __init__(self, points, group_threshold=15, depth=1):
        self.child_left = self.child_right = None
        self.points = points
        self.group_threshold = group_threshold
        self.num_points = self.points.shape[0]
        self.split(self.points, depth)

    def __str__(self):
        return f"<{self.__class__.__name__} num_points:{self.num_points}>"

    @property
    def is_leaf(self):
        return self.child_left == None and self.child_right == None

    def split(self, points, depth=1):
        plane, points_left, points_right = self.get_split(points)
        if not plane or depth >= 50:
            return self
        self.splitting_plane = plane
        self.child_left = HyperplaneCollection(points_left, depth=depth + 1)
        self.child_right = HyperplaneCollection(points_right, depth=depth + 1)

    def decision_function(self, points):
        return np.array([self.get_depth(point) for point in points])

    def get_depth(self, point):
        if self.is_leaf:
            return 1
        else:
            if self.splitting_plane.point_relative_to_plane(point) < 0:
                return 1 + self.child_left.get_depth(point)
            else:
                return 1 + self.child_right.get_depth(point)

    def get_split(self, points):
        if self.num_points < 2:
            return (None, None, None)
        
        splitting_plane = generate_splitting_plane(points)
        positions = splitting_plane.position_of_points(points)
        #split_point = np.random.uniform(low=np.min(positions), high=np.max(positions))
        split_point = np.dot(splitting_plane.normal, splitting_plane.origin)
        positions_x = np.array([splitting_plane.point_relative_to_plane_x(point) for point in points])
        split_point_x = np.random.uniform(low=np.min(positions), high=np.max(positions))
        #positions_x = positions_x - split_point
        #print(positions-positions_x)
        #positions -= split_point
        points_left = points[np.where(positions < split_point)]
        points_right = points[np.where(positions > split_point)]
        return (splitting_plane, points_left, points_right)


class Hyperplane(object):
    def __init__(self, origin, normal):
        self.origin = origin
        self.normal = normal

    def point_relative_to_plane(self, point):
        result = np.dot(self.normal, point) - np.dot(self.normal, self.origin)
        result = np.dot(self.normal, point)
        return result

    def point_relative_to_plane_x(self, point):
        result = np.dot(self.normal, point)
        return result

    def position_of_points(self, points):
        offset = np.dot(self.normal, self.origin)
        position = np.array([self.point_relative_to_plane(point) for point in points])
        #position_x = np.array([self.point_relative_to_plane_x(point)-offset for point in points])
        #print(position - position_x)
        return position

def generate_splitting_plane(points):
    #feature_ranges = get_feature_ranges(points)
    origin = np.fromiter(generate_point(points), dtype=float)
    normal = np.fromiter(generate_point(points), dtype=float)
    #origin = np.zeros_like(normal)
    #normal -= origin
    return Hyperplane(origin=origin, normal=normal)
    
def generate_point(points):
    """ Generat an n-dimensional normal vector

    For now just do so by sampling the points from a uniform distribution.
    """
    feature_mins_maxes = get_feature_ranges(points)
    for min_, max_, in get_feature_ranges(points):
        yield np.random.uniform(low=min_, high=max_)

In [38]:
N_ESTIMATORS = 1
SCORE_AT = 2.5

n = 1000 # number of entries
p = 2    # features

infection_pct = 0.05
X, y = _gen_easy_data(n, p, infection_pct)

scores_r, depths_r, y_pred_r, y_r = run_plane_simul(X, y)
print("\nDone plane simul-----\n")
scores_i, depths_i, y_pred_i, y_i = run_iforest_simul(X, y)

Beginning plane fit...
done fitting
Average anomalous depth: 2.58
Average non-anomalous depth: 3.28736842105

Done plane simul-----

Beginning iforest fit...
done fitting
Average anomalous depth: 8.62
Average non-anomalous depth: 19.3263157895


In [21]:
X[np.where(y==1.0)]

array([[ -1.37280703,  -0.56024005],
       [ 11.36570884,   3.77988007],
       [  3.34012657,  13.34362564],
       [  4.02681731,  11.14744039],
       [ 17.42444546,  11.60953577],
       [ -9.08602304, -12.92151409],
       [ 15.14425805,  12.70707588],
       [ -3.39544144,   6.9006884 ],
       [  1.07552752,  11.03316679],
       [ -6.8240313 ,   9.86519819],
       [ 16.71098235,  -8.28630956],
       [ -2.11746045,   1.40887617],
       [  5.39372271, -16.69124418],
       [  2.16933758,  28.97520456],
       [-24.94376518,  11.00170154],
       [  1.23544578,   4.59481833],
       [  3.69013028,  10.72801301],
       [ -6.18609373,  -3.05111084],
       [  8.22573896,  12.85200319],
       [  4.95286347,  -7.01886689],
       [ 15.40701759,  13.5103146 ],
       [  6.69622409,  22.20154637],
       [  0.19989231,   7.20879773],
       [ 17.38344867,  -4.9211051 ],
       [  5.56288721,   7.45250685],
       [  3.53438398,  10.14586437],
       [  5.13897434,  -3.63779169],
 

In [22]:
rhp = RandomHyperplanes(n_estimators=N_ESTIMATORS)
rhp = rhp.fit(X)
depths = rhp.get_depths(X)

In [23]:
print(np.unique(depths))

[  2.   4.   6.  10.  18.  20.  21.  25.  29.  33.  34.  35.  37.  38.  39.
  40.  42.  44.  46.  48.  50.]
